# Example for usage of tools and configurations
This notebook is an example of how simulation tools, configurations, and reporting can be done harmoniously if a common structure is established. <br>
Note that this structure is commonly seen among large software teams that are constantly co-developing and producing/maintaining production code. <br>

**This notebook represents a report that could be the artifact of an analysis informing science impact of a given parameter.**

## Load dependencies

In [ ]:
# load dependencies for this report
import pathlib
import tomlkit
import git
import matplotlib.pyplot as plt

from utils_config import ConfigLoader  # Used to load and parse configs

## Import simulation Tool(s)

In [ ]:
import example_tool

Display the git hash being executed so there is traceability for reproducing the result in the future. <br>
If it was a proper package, then one would just run the following: <br>
`print example_tool.__version__` <br>
Because this is an example a local file is used, so in this case we'll show the current git hash.


In [ ]:
path_to_example_tool=pathlib.Path('.') 
repo = git.Repo(path_to_example_tool,search_parent_directories=True)
print(f'Repo located at: {repo.common_dir}')
print(f'Has the repo been modified? {repo.is_dirty()}')
print(f'Repo is at commit: {repo.commit()}')
print(f'Active branch is: {repo.active_branch}')

## Load the configuration(s)
Configurations are be stored in external packages so everyone can access them. They can also be used by CI processes that can automatically generate new reports or values and perform analyses when changes to inputs are performed.<br>
Configurations could be stored inside packages (which can be imported) or potentially as cloned repositories. <br>
The case below imports the configs as python packages, which have been cloned and pip installed.

In [ ]:
import config_project_template

data=config_project_template.load_config_values('unitless')
print(data)


We can also import other configurations and draw from them

In [ ]:
import config_stp
data_stp = config_stp.load_config_values('unitless')

The installed version of the package can be found by printing the version.<br>
Having these displayed in the analysis report aids in the reproduction of the results at future times.

In [ ]:
print(f'{config_project_template.__version__=}')
print(f'{config_stp.__version__=}')

## Begin Analysis

In this case, we consider the example where we want to iterate over a configuration value and examine the result<br>
In this example let's consider changing the slew speed and we'll run that through our analysis tool see how that affects how long to get to and acquire a target.

In [ ]:
print(f'The baseline slew speed is {data_stp["observatory"]["motion"]["slew_avg_speed"]}')

speeds = [0.1,0.2,0.5,0.6,0.8,1,2,4,6,10] # [Celsius]

slew_distance = 50 # degrees


Instantiate example tool class

In [ ]:
example_class=example_tool.exampleTool()

Loop over the variable being iterated upon and modify the configuration locally

In [ ]:
results={'speed':[],'sim_results':[],'config':[]}
for s in speeds:
    data_stp["observatory"]["motion"]["slew_avg_speed"]=s
    run_result = example_class.run_sim(slew_distance,config=data_stp)
    results['config'].append(data_stp)
    results['speed'].append(s)
    results['sim_results'].append(run_result)

## Perform some analysis
For example, one could make a plot and look at the results

In [ ]:
x=results['speed']
y=results["sim_results"]
plt.plot(x,y,'x')
plt.ylabel('time to target [s]')
plt.xlabel('slew speed [deg/s]')

## Report the take-home points in an obvious manner

In [ ]:
print(f"The  acquisition overhead starts to dominate over slew speeds when the speed is ~1.5 degrees per second")